In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import torch
import h5py
import os
import sys
import scipy
import damselfly as df
import scipy.signal
import scipy.stats
import scipy.integrate 
import json

PATH = '/storage/home/adz6/group/project'
RESULTPATH = os.path.join(PATH, 'results/damselfly')
PLOTPATH = os.path.join(PATH, 'plots/damselfly')
DATAPATH = os.path.join(PATH, 'damselfly/data/datasets')
SIMDATAPATH = os.path.join(PATH, 'damselfly/data/sim_data')
TRAINPATH = os.path.join(PATH, 'damselfly/training')

"""
Date: 6/25/2021
Description: template
"""

def InceptionConv(n_in_filter, n_out_filter, filter_sizes, ):

    conv_layers = []

    for size in filter_sizes:
        conv_layers.append(df.models.ConvRelu(n_in_filter, n_out_filter, size, 1))

    return conv_layers

class InceptionModule(torch.nn.Module):

    def __init__(self, n_in_filter, n_out_filter, filter_sizes,):
        
        super(InceptionModule, self).__init__()

        self.n_in_filter = n_in_filter
        self.n_out_filter = n_out_filter
        self.filter_sizes = filter_sizes

        self.conv_layer_list = InceptionConv(self.n_in_filter, self.n_out_filter, self.filter_sizes)

    def forward(self, x):

        output_list = []
        for conv_layer in self.conv_layer_list:
            output_list.append(conv_layer(x))

        return torch.cat(output_list, 1)

def InceptionStack(in_filter_list, out_filter_list, filter_sizes, maxpool_kernel_sizes):
    
    stack_list = []
    
    for i in range(len(in_filter_list)):
        stack_list.append(InceptionModule(in_filter_list[i], out_filter_list[i], filter_sizes, ))
        stack_list.append(torch.nn.MaxPool1d(maxpool_kernel_sizes[i]))
        
    return torch.nn.Sequential(*stack_list)

In [ ]:
x = torch.rand((10, 2, 8192))

In [ ]:
inception_module = InceptionModule(2, 10, [4, 8, 16, 32, 64])

In [ ]:
inception_module(x).shape

In [ ]:
in_filter_list = [2, 4 * 10, 4 * 10]
out_filter_list = [10, 10, 10] # number of filters per kernel size
kernel_sizes = [4, 8, 16, 32]
maxpool_kernel_sizes = [4, 4, 4]


inception_layers = InceptionStack(in_filter_list, out_filter_list, kernel_sizes, maxpool_kernel_sizes)

In [ ]:
inception_layers(x).shape

In [ ]:
inception_layers

In [ ]:
conv_list_1 = [
        [
            [2,],
            [10,],
            [4,],
            [1], # dilation
            4
        ],
    ]

conv_list_2 = [
        [
            [2,],
            [10,],
            [8,],
            [1], # dilation
            4
        ],
    ]


conv1 = df.models.ConvStack1D(conv_list_1)
conv2 = df.models.ConvStack1D(conv_list_2)

In [ ]:
conv1(x).shape


In [ ]:
conv2(x).shape

In [ ]:
x = torch.rand((2, 2, 4))

In [ ]:
torch.cat([x, x]).shape

In [ ]:
64 + 128 + 32 + 32

In [ ]:
8192 / 4

In [ ]:
8192 / 4 + 8192 / 8 + 8192 / 16

In [ ]:
8192 / 4 + 8192 / 8 + 8192 / 16 + 8192 / 32 + 8192 / 64 + 8192 / 128 + 8192 / 256 + 8192 / 512 + 8192 / 1024

In [ ]:
1024 * 8192 / 4

In [ ]:
(512 * 8192 / 4) + (512 * 8192) / 8

In [ ]:
(256 * 8192 / 4) + (256 * 8192) / 16 + (256 * 8192 / 64) + (256 * 8192) / 256 + (256 * 8192) / 1024

In [ ]:
(128 * 8192 / 4) + (128 * 8192) / 8 + (128 * 8192 / 16) + (128 * 8192) / 32 + (128 * 8192 / 64) + (128 * 8192) / 128 + (128 * 8192 / 256) + (128 * 8192) / 512

In [ ]:
20 * 10 * 2048

In [ ]:
10 * 10 * (2048 + 1024)

In [ ]:
x = torch.rand((10, 2, 8192))


nch = 2
nslice = 1
input_shape_1d = 8192
conv_list_1d = [
        [
            [nch * nslice, 4, 4],
            [4, 4,4],
            [16, 16, 16],
            [1, 1, 1], # dilation
            16
        ],
        [
            [4, 6, 6],
            [6, 6, 6],
            [8, 8, 8],
            [1, 1, 1],
            8
        ],
        [
            [6, 8, 8],
            [8, 8, 8],
            [4, 4, 4],
            [1, 1, 1],
            4
        ],
    ]

model_config_1d_cnn = {
    'nclass': 2,
    'nch': 2,
    'conv': conv_list_1d
    }

linear_list_1d = [
        [df.models.GetConv1DOutputSize(model_config_1d_cnn['conv'], model_config_1d_cnn['nch'], input_shape_1d) + 10, 100],
        [100, 20],
        [0.0, 0.0]
    ]

model = DFHybrid(model_config_1d_cnn['nclass'], 
        model_config_1d_cnn['nch'], 
        model_config_1d_cnn['conv'], 
        linear_list_1d,
                 x
                )

In [ ]:
y = torch.rand(5, 2, 8192)

print(model(y).shape)

In [ ]:
y = MFConv(x)
print(y.shape)

In [ ]:
n_pc = 256

conv_list = [
    [
        [2, 20, 20],
        [20, 20, 20],
        [(n_pc, 4), (n_pc, 4), (n_pc, 4)],
        (1, 4)
    ],
    [
        [20, 40, 40],
        [40, 40, 40],
        [(n_pc, 4), (n_pc, 4), (n_pc, 4)],
        (1, 4)
    ],
    [
        [40, 80, 80],
        [80, 80, 80],
        [(n_pc, 4), (n_pc, 4), (n_pc, 4)],
        (1, 4)
    ],
]

in_ch = 2
input_shape = (256, 128)

GetConv2DOutputSize(conv_list, in_ch, input_shape)



In [ ]:
x = torch.rand((1, 2, 60, 8192))

layer2d = torch.nn.Conv2d(2, 20, (60, 12), padding='same')

maxlayer = torch.nn.MaxPool2d((1, 16))

In [ ]:
layer2d(x).shape

In [ ]:
maxlayer(layer2d(x)).shape